# Inicialização

In [1]:
import  torch
import  torch.nn as nn
import  torch.nn.functional as F
import  torch.optim as optim
import  psutil
import  os
import joblib
import  warnings
import  pandas as pd
import  seaborn as sns
import  numpy as np
import  matplotlib.pyplot as plt
import  time

import imodelsx
from imodelsx import KANClassifier


from    sklearn.model_selection import train_test_split
from    sklearn.metrics import confusion_matrix, classification_report, accuracy_score, f1_score
from    sklearn.model_selection import ParameterGrid
from    sklearn.ensemble import RandomForestClassifier
from    sklearn.preprocessing import LabelEncoder
from    sklearn.model_selection import GridSearchCV
from    sklearn.preprocessing import StandardScaler
from    sklearn.preprocessing import MinMaxScaler
from    sklearn.datasets import make_classification, make_regression

from    imblearn.over_sampling import SMOTE
from    imblearn.under_sampling import RandomUnderSampler
from    imblearn.pipeline import Pipeline
from    imblearn.combine import SMOTETomek

warnings.filterwarnings("ignore")

if torch.cuda.is_available():
  device = torch.device("cpu")
else:
  device = torch.device("cpu")

print(device)

cpu


# Carregar e Preparar Conjunto de Dados

In [2]:
# dados importados da plataforma KAGGLE - https://www.kaggle.com/datasets/shivamb/machine-predictive-maintenance-classification
df = pd.read_csv("predictive_maintenance.csv")
df.head()


,UDI,Product ID,Type,Air temperature [K],Process temperature [K],Rotational speed [rpm],Torque [Nm],Tool wear [min],Target,Failure Type
0,1,M14860,M,298.1,308.6,1551,42.8,0,0,No Failure
1,2,L47181,L,298.2,308.7,1408,46.3,3,0,No Failure
2,3,L47182,L,298.1,308.5,1498,49.4,5,0,No Failure
3,4,L47183,L,298.2,308.6,1433,39.5,7,0,No Failure
4,5,L47184,L,298.2,308.7,1408,40.0,9,0,No Failure


* descartar as colunas desnecessárias

In [3]:
df=df.drop(['Product ID','UDI','Target'],axis=1)

* codificar a coluna de falhas para tipo int entre 0 e 5

In [4]:
label_encoder = LabelEncoder()

# Perform label encoding
df['Failure Type_encoded'] = label_encoder.fit_transform(df['Failure Type'])

df['Failure Type_encoded'].value_counts()

Failure Type_encoded
1    9652
0     112
3      95
2      78
5      45
4      18
Name: count, dtype: int64

* converter a coluna 'Type' (string) em outras duas separadas do tipo bool (uma para cada tipo de material) 

In [5]:
df = pd.get_dummies(df, columns=['Type'], prefix='Type', drop_first=True)
df.head()


,Air temperature [K],Process temperature [K],Rotational speed [rpm],Torque [Nm],Tool wear [min],Failure Type,Failure Type_encoded,Type_L,Type_M
0,298.1,308.6,1551,42.8,0,No Failure,1,False,True
1,298.2,308.7,1408,46.3,3,No Failure,1,True,False
2,298.1,308.5,1498,49.4,5,No Failure,1,True,False
3,298.2,308.6,1433,39.5,7,No Failure,1,True,False
4,298.2,308.7,1408,40.0,9,No Failure,1,True,False


* separar os dados em: dados de entrada (features), X  e dados de saída (label), y

In [6]:
# Definição do conjunto de dados de entrada e do conjunto de dados de saída

X_raw = df.drop(['Failure Type','Failure Type_encoded'], axis=1).values
y_raw= df['Failure Type_encoded'].values


* Em aplicações do mundo real, a modelagem de classificação frequentemente enfrenta o problema de conjuntos de dados desequilibrados, onde o número de instâncias da classe majoritária é muito maior do que o da classe minoritária, o que dificulta o aprendizado adequado do modelo em relação à classe minoritária. Isso se torna um problema sério quando a informação contida na classe minoritária é mais importante como, por exemplo, no conjunto utilizado.

* Uma das abordagens populares para resolver o problema de conjuntos de dados desequilibrados é a superamostragem da classe minoritária ou a subamostragem da classe majoritária. No entanto, essas abordagens possuem suas próprias limitações. No método tradicional de superamostragem, a ideia é duplicar aleatoriamente alguns exemplos da classe minoritária — essa técnica não adiciona novas informações ao conjunto de dados. Por outro lado, o método de subamostragem é realizado removendo aleatoriamente alguns exemplos da classe majoritária, o que resulta na perda de algumas informações originais dos dados.

In [7]:
print('Total de amostras antes balanceamento:',df['Failure Type_encoded'].count())
print('Total de amostras por falha:\n\n', df['Failure Type_encoded'].value_counts())

Total de amostras antes balanceamento: 10000
Total de amostras por falha:

 Failure Type_encoded
1    9652
0     112
3      95
2      78
5      45
4      18
Name: count, dtype: int64


* Uma solução para superar essas limitações é gerar novos exemplos sintetizados a partir da classe minoritária existente. Esse método é conhecido como Técnica de Superamostragem da Minoria Sintética (SMOTE). Existem muitas variações do SMOTE, mas neste artigo será explicado o método SMOTE-Tomek Links e sua implementação em Python. Esse método combina a superamostragem do SMOTE com a subamostragem dos Tomek Links.

* O método SMOTE-Tomek Links, desenvolvido por Chawla et al. (2002) é uma combinação de técnicas que visa equilibrar os dados ao aumentar a representatividade da classe minoritária através da criação de novos exemplos sintéticos e ao mesmo tempo remover exemplos que são considerados ruidosos ou redundantes na classe majoritária. A implementação desta abordagem utilizando Python é apresentada, detalhando os passos e a lógica do algoritmo, assim como os resultados obtidos em diferentes conjuntos de dados desequilibrados.

* A combinação das técnicas de superamostragem e subamostragem, especificamente utilizando o método SMOTE-Tomek Links, mostra-se eficaz na melhoria do desempenho dos modelos de classificação em conjuntos de dados desequilibrados. Esta abordagem permite ao modelo aprender de forma mais eficiente a partir da classe minoritária, mantendo a integridade e a diversidade da informação no conjunto de dados.



In [8]:
#implementação do método SMOTE no conunto de dados
smt=SMOTETomek(sampling_strategy='auto',random_state=42)
X_resampled, y_resampled = smt.fit_resample(X_raw, y_raw)

# Convertendo para dataframe do pandas
df_resampled = pd.DataFrame(X_resampled, columns=[f'feature_{i}' for i in range(X_resampled.shape[1])])
df_resampled['Type_encoded'] = y_resampled 

df_resampled.head()

#distribuição das classes após balanceamento
print('Total de amostras após balanceamento:',df_resampled['Type_encoded'].count())
print('Total de amostras por falha:\n\n', df_resampled['Type_encoded'].value_counts())

Total de amostras após balanceamento: 57868
Total de amostras por falha:

 Type_encoded
2    9652
0    9649
5    9647
3    9645
4    9645
1    9630
Name: count, dtype: int64


In [9]:
df_resampled = df_resampled.rename(columns={'feature_0':'Temp_Ar'})
df_resampled = df_resampled.rename(columns={'feature_1':'Temp_Pr'})
df_resampled = df_resampled.rename(columns={'feature_2':'Vel_Spindle'})
df_resampled = df_resampled.rename(columns={'feature_3':'Torque'})
df_resampled = df_resampled.rename(columns={'feature_4':'Desg_Ferr'})
df_resampled = df_resampled.rename(columns={'feature_5':'Mat_L'})
df_resampled = df_resampled.rename(columns={'feature_6':'Mat_M'})

df_resampled = df_resampled.rename(columns={'Type_encoded':'Tipo_Falha'})

df_resampled.head()


,Temp_Ar,Temp_Pr,Vel_Spindle,Torque,Desg_Ferr,Mat_L,Mat_M,Tipo_Falha
0,298.1,308.6,1551.0,42.8,0.0,0.0,1.0,1
1,298.2,308.7,1408.0,46.3,3.0,1.0,0.0,1
2,298.1,308.5,1498.0,49.4,5.0,1.0,0.0,1
3,298.2,308.6,1433.0,39.5,7.0,1.0,0.0,1
4,298.2,308.7,1408.0,40.0,9.0,1.0,0.0,1


In [10]:
X_ = df_resampled.drop(['Tipo_Falha'], axis="columns")
scaler = MinMaxScaler(feature_range=(-1, 1))

X   = scaler.fit_transform(X_)    
joblib.dump(scaler, 'minmax_scaler.pkl')

y = df_resampled['Tipo_Falha']


In [34]:
def grid_search(X_train, y_train, X_test, y_test, param_grid, device):

    results = []
    i = 0
    for h_layer_size in param_grid['hidden_layer_size']:
        for r_activation in param_grid['regularize_activation']:
            for r_entropy in param_grid['regularize_entropy']:
                for r_ridge in param_grid['regularize_ridge']:
                    for b_size in param_grid['batch_size']:
                        for w_decay in param_grid['weight_decay']:
                            for g in param_grid['gamma']:
                                for l in param_grid['lr']:
                                    model = KANClassifier(hidden_layer_size     = h_layer_size,
                                                          device                = device,
                                                          regularize_activation = r_activation,
                                                          regularize_entropy    = r_entropy,
                                                          regularize_ridge      = r_ridge) 
                                    i+=1  
                                    progress = round(100*i/6561,2)

                                    print('------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------\n')

                                    print(f'Teste número: {i}')
                                    print(f'Progresso: {progress} %')
                                    print(f'Parâmetros: {h_layer_size} | activation: {r_activation} | entropy: {r_entropy} | ridge: {r_ridge} | batch size: {b_size} | decay: {w_decay} | gamma: {g} | learning reate: {l}')
                                    
                                    start_train = time.time()

                                    model.fit(X_train,
                                              y_train,
                                              batch_size = b_size,
                                              lr = l,
                                              weight_decay = w_decay,
                                              gamma = g)
                                    
                                    end_train = time.time()

                                    y_train_pred = model.predict(X_train)
                                    f1_train = f1_score(y_train, y_train_pred, average='weighted')

                                    y_test_pred = model.predict(X_test)
                                    f1_test = f1_score(y_test, y_test_pred, average='weighted')

                                    acc = accuracy_score(y_test, y_test_pred)
                                    print(f'Acuracidade: {acc}')

                                    total_train_time = end_train-start_train

                                    result = {
                                              'ID'                      : i,
                                              'train_time'              : total_train_time,
                                              'accuracy'                : acc,
                                              'f1_train'                : f1_train,
                                              'f1_test'                 : f1_test,
                                              'layers'                  : h_layer_size,
                                              'regularize_activation'   : r_activation,
                                              'regularize_entropy'      : r_entropy,
                                              'regularize_ridge'        : r_ridge ,
                                              'batch_size'              : b_size,
                                              'weight_decay'            : w_decay ,
                                              'gamma'                   : g,
                                              'learning'                : l
                                              }

                                    results.append(result)   
    return results


In [35]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [36]:
param_grid = {
    'hidden_layer_size'     : [256, 512, 1024],
    'regularize_activation' : [0.4, 0.5, 0.6],
    'regularize_entropy'    : [0.4, 0.5, 0.6], 
    'regularize_ridge'      : [0.05, 0.1, 0.2], 
    'batch_size'            : [64, 128, 512],    
    'weight_decay'          : [0.005, 0.01, 0.03],
    'gamma'                 : [0.4, 0.5, 0.6],
    'lr'                    : [0.05, 0.07, 0.1]
    }


grid_results = grid_search(X_train, y_train, X_test, y_test, param_grid, device)

best_result_Accuracy        = max(grid_results, key=lambda x: x['accuracy'])
best_result_F1_Score_train  = max(grid_results, key=lambda x: x['f1_train'])
best_result_F1_Score_test   = max(grid_results, key=lambda x: x['f1_test'])

print


------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

Teste número: 1
Progresso: 0.02 %
Parâmetros: 256 | activation: 0.4 | entropy: 0.4 | ridge: 0.05 | batch size: 64 | decay: 0.005 | gamma: 0.4 | learning reate: 0.05


  7%|▋         | 7/100 [00:29<06:38,  4.28s/it]

	Early stopping


0.955590115776741
------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

Teste número: 2
Progresso: 0.03 %
Parâmetros: 256 | activation: 0.4 | entropy: 0.4 | ridge: 0.05 | batch size: 64 | decay: 0.005 | gamma: 0.4 | learning reate: 0.07


  5%|▌         | 5/100 [00:22<07:06,  4.49s/it]

...

0.9427164333851736
------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

Teste número: 6
Progresso: 0.09 %
Parâmetros: 256 | activation: 0.4 | entropy: 0.4 | ridge: 0.05 | batch size: 64 | decay: 0.005 | gamma: 0.5 | learning reate: 0.1


  6%|▌         | 6/100 [00:28<07:19,  4.67s/it]

	Early stopping


0.9442716433385173
------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

Teste número: 7
Progresso: 0.11 %
Parâmetros: 256 | activation: 0.4 | entropy: 0.4 | ridge: 0.05 | batch size: 64 | decay: 0.005 | gamma: 0.6 | learning reate: 0.05


  5%|▌         | 5/100 [00:22<06:46,  4.28s/it]